In [20]:
'''Import libraries'''
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
import wandb
import torch
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset
import os
from torch.utils.data import Dataset
import torch
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score
from imblearn.under_sampling import RandomUnderSampler
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizerFast, BertForSequenceClassification, AdamW, Trainer, TrainingArguments
from tqdm import tqdm
from torch.nn import functional as F
import torch.nn as nn

wandb.login()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [21]:
'''Variables and parameters'''

SAMPLES_TO_TRAIN=5000

N_LABELS=2
MAX_LEN = 256
EPOCHS=100
PATIENCE=10
LEARNING_RATE=.00005
WEIGHT_DECAY=.01
BATCH_SIZE=64
METRIC_FOR_BEST_MODEL='eval_loss'
if METRIC_FOR_BEST_MODEL=='eval_loss':
    GREATER_IS_BETTER = False
else:
    GREATER_IS_BETTER = True

In [22]:
'''Preparing dataset'''

df = pd.read_json(os.getcwd()+'/datasets/subtaskA_train_monolingual.jsonl', lines=True)
df = df[['text', 'label']]

train_df=df.sample(round(SAMPLES_TO_TRAIN))
test_train_df=df.sample(round(SAMPLES_TO_TRAIN*.2))

df = pd.read_json(os.getcwd()+'/datasets/subtaskA_dev_monolingual.jsonl', lines=True)
df = df[['text', 'label']]

val_df= df.sample(round(SAMPLES_TO_TRAIN*.2))
test_dev_df= df.sample(round(SAMPLES_TO_TRAIN*.2))

# we balance the training set
print(f'Dataset size before balancing: {train_df.shape}')
counts = train_df['label'].value_counts()
sampler = RandomUnderSampler(random_state=42)
x_text, y = sampler.fit_resample(train_df[['text']], train_df['label'])

print(f'Dataset size after balancing: {x_text.shape}')
print(f'Entried dropped: {train_df.shape[0]-x_text.shape[0]}')

# Create a new balanced DataFrame
train_df = pd.DataFrame({'text': x_text['text'], 'label': y})

# Print the balanced DataFrame
print("\nBalanced DataFrame:")
print(train_df['label'].value_counts())

Dataset size before balancing: (5000, 2)
Dataset size after balancing: (4736, 1)
Entried dropped: 264

Balanced DataFrame:
label
0    2368
1    2368
Name: count, dtype: int64


In [4]:
'''loading glove'''
embeddings_index={}
with open('../OtherData/glove.6B.200d.txt','r',encoding='utf-8') as f:
    for line in f:
        values=line.split()
        word=values[0]
        vectors=np.asarray(values[1:],'float32')
        embeddings_index[word]=vectors
f.close()
print('Found %s word vectors.' % len(embeddings_index))


Found 400000 word vectors.


In [23]:
'''glove building'''

from nltk.tokenize import word_tokenize
from tqdm import tqdm 

def sent2vec(s):
    """ Function Creates a normalized vector for the whole sentence"""
    words = str(s).lower()
    words = word_tokenize(words)
    words = [w for w in words if w.isalpha()]
    M = []
    for w in words:
        try:
            M.append(embeddings_index[w])
        except:
            continue
    M = np.array(M)
    v = M.sum(axis=0)
    if type(v) != np.ndarray:
        return np.zeros(200)
    return v / np.sqrt((v ** 2).sum())

print('Training df:')
train_x = np.array([sent2vec(x) for x in tqdm(train_df['text'])])
print(train_x.shape)
train_y=train_df['label']

print('\nValidation df:')
val_x = np.array([sent2vec(x) for x in tqdm(val_df['text'])])
print(val_x.shape)
val_y=val_df['label']

print('\nTesting from training dataset df:')
test_train_x = np.array([sent2vec(x) for x in tqdm(test_train_df['text'])])
print(test_train_x.shape)
test_train_y=test_train_df['label']

print('\nTesting from dev dataset df:')
test_dev_x = np.array([sent2vec(x) for x in tqdm(test_dev_df['text'])])
print(test_dev_x.shape)
test_dev_y=test_dev_df['label']


Training df:


100%|██████████| 4736/4736 [00:07<00:00, 639.50it/s]


(4736, 200)

Validation df:


100%|██████████| 1000/1000 [00:01<00:00, 885.47it/s]


(1000, 200)

Testing from training dataset df:


100%|██████████| 1000/1000 [00:01<00:00, 643.95it/s]


(1000, 200)

Testing from dev dataset df:


100%|██████████| 1000/1000 [00:01<00:00, 833.59it/s]

(1000, 200)


In [24]:
'''Preparing for training'''

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Initialize the StandardScaler
scaler = StandardScaler()
# Fit the scaler to the training data and transform the data
train_x = scaler.fit_transform(train_x)
val_x = scaler.transform(val_x)  # Note that we use the same scaler to transform the test data
test_train_x = scaler.transform(test_train_x)
test_dev_x = scaler.transform(test_dev_x)
# If this scaler is going to be used later on for prediction it must be saved, for example with pickle
import pickle

# Save the trained scaler
with open('scaler.pkl', 'wb') as scaler_file:
    pickle.dump(scaler, scaler_file)

In [12]:
'''Dense Network'''

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Dense, Activation, Dropout, BatchNormalization, SimpleRNN
from tensorflow.keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from tensorflow.keras.preprocessing import sequence, text
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import backend as K

def f1_score(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

model = Sequential()

model.add(Dense(200, input_dim=train_x.shape[1], activation='relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Dense(100, activation='relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Dense(100, activation='relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Dense(1))
model.add(Activation('sigmoid'))
# compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy','Recall','Precision','AUC',f1_score])
model.summary()

stop_early = EarlyStopping(monitor='val_loss', patience=20)
model.fit(train_x, y=train_y, batch_size=64, 
              epochs=1000, verbose=1, 
              validation_data=(test_train_x, test_train_y),
              callbacks=[stop_early])

metrics=model.evaluate(test_train_x, test_train_y)


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 200)               40200     
                                                                 
 dropout (Dropout)           (None, 200)               0         
                                                                 
 batch_normalization (Batch  (None, 200)               800       
 Normalization)                                                  
                                                                 
 dense_1 (Dense)             (None, 100)               20100     
                                                                 
 dropout_1 (Dropout)         (None, 100)               0         
                                                                 
 batch_normalization_1 (Bat  (None, 100)               400       
 chNormalization)                                       

In [18]:
loss, accuracy, recall, precision, auc, f1 = metrics

print(f"Accuracy: {accuracy}")
print(f"Recall: {recall}")
print(f"Precision: {precision}")
print(f"AUC: {auc}")
print(f"F1 Score: {f1}")

Accuracy: 0.7350000143051147
Recall: 0.6595744490623474
Precision: 0.7469879388809204
AUC: 0.8143817782402039
F1 Score: 0.686847984790802


In [17]:
metrics=model.evaluate(val_x, val_y)

63/63 [==============================] - 0s 797us/step - loss: 1.9690 - accuracy: 0.4950 - recall: 0.2084 - precision: 0.5145 - auc: 0.4828 - f1_score: 0.2872


In [18]:
loss, accuracy, recall, precision, auc, f1 = metrics

print(f"Accuracy: {accuracy}")
print(f"Recall: {recall}")
print(f"Precision: {precision}")
print(f"AUC: {auc}")
print(f"F1 Score: {f1}")

Accuracy: 0.4950000047683716
Recall: 0.20841486752033234
Precision: 0.5144927501678467
AUC: 0.48282068967819214
F1 Score: 0.2871707081794739


In [25]:
'''creating custom dataset'''
class CustomDataset(Dataset):
    def __init__(self, features, labels):
        self.features = features
        self.labels = labels

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        return {
            'x': self.features[idx],  # changed 'features' to 'x'
            'label': self.labels[idx]  # changed 'labels' to 'label'
        }

# Verify the shape and type of the data
print(train_x.shape, train_y.shape, val_x.shape, val_y.shape)
print(test_train_x.shape, test_train_y.shape, test_dev_x.shape, test_dev_y.shape)

# Convert numpy arrays to PyTorch tensors
train_x = torch.tensor(train_x, dtype=torch.float32)
val_x = torch.tensor(val_x, dtype=torch.float32)

test_train_x = torch.tensor(test_train_x, dtype=torch.float32)
test_dev_x = torch.tensor(test_dev_x, dtype=torch.float32)


# now y
train_y = torch.tensor(train_y.to_numpy(), dtype=torch.float32) if isinstance(train_y, pd.Series) else torch.tensor(train_y, dtype=torch.float32)
val_y = torch.tensor(val_y.to_numpy(), dtype=torch.float32) if isinstance(val_y, pd.Series) else torch.tensor(val_y, dtype=torch.float32)

test_train_y = torch.tensor(test_train_y.to_numpy(), dtype=torch.float32) if isinstance(test_train_y, pd.Series) else torch.tensor(test_train_y, dtype=torch.float32)
test_dev_y = torch.tensor(test_dev_y.to_numpy(), dtype=torch.float32) if isinstance(test_dev_y, pd.Series) else torch.tensor(test_dev_y, dtype=torch.float32)

# Create dataset and dataloaders
train_dataset = CustomDataset(train_x, train_y)
val_dataset = CustomDataset(val_x, val_y)
test_train_dataset = CustomDataset(test_train_x, test_train_y)
test_dev_dataset = CustomDataset(test_dev_x, test_dev_y)

# Check the length of datasets
print(len(train_dataset), len(val_dataset))
print(len(test_train_dataset), len(test_dev_dataset))

(4736, 200) (4736,) (1000, 200) (1000,)
(1000, 200) (1000,) (1000, 200) (1000,)
4736 1000
1000 1000


In [26]:
'''building custom model'''
class MyModel(nn.Module):
    def __init__(self, input_dim):
        super(MyModel, self).__init__()
        self.fc1 = nn.Linear(input_dim, 200)
        self.dropout1 = nn.Dropout(0.2)
        self.batchnorm1 = nn.BatchNorm1d(200)
        self.fc2 = nn.Linear(200, 100)
        self.dropout2 = nn.Dropout(0.2)
        self.batchnorm2 = nn.BatchNorm1d(100)
        self.fc3 = nn.Linear(100, 100)
        self.dropout3 = nn.Dropout(0.2)
        self.batchnorm3 = nn.BatchNorm1d(100)
        self.fc4 = nn.Linear(100, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x, labels=None):
        x = F.relu(self.batchnorm1(self.dropout1(self.fc1(x))))
        x = F.relu(self.batchnorm2(self.dropout2(self.fc2(x))))
        x = F.relu(self.batchnorm3(self.dropout3(self.fc3(x))))
        logits = self.fc4(x)
        outputs = self.sigmoid(logits)

        if labels is not None:
            loss = F.binary_cross_entropy(outputs, labels.unsqueeze(-1))
            return loss, outputs
        return outputs

# Instantiate the model
model = MyModel(input_dim=train_x.shape[1])

In [27]:
'''metrics'''

def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    labels = p.label_ids
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    auc = roc_auc_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'auc': auc,
        'precision': precision,
        'recall': recall,
    }

In [28]:
'''Define training arguments and initialize trainer'''

from transformers import EarlyStoppingCallback

BATCH_SIZE=100

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    learning_rate=LEARNING_RATE,
    warmup_steps=500,
    weight_decay=WEIGHT_DECAY,
    metric_for_best_model=METRIC_FOR_BEST_MODEL,
    greater_is_better=GREATER_IS_BETTER,
    logging_dir='./logs',
    logging_steps=1500,
    do_train=True,
    do_eval=True,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    push_to_hub=False,
    logging_first_step=False,
    load_best_model_at_end=True,
    save_total_limit=2,
    report_to="wandb"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_train_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=PATIENCE)]
)

In [29]:
trainer.train()

  1%|          | 39/4800 [00:00<00:12, 381.94it/s]c:\Users\Ghiki\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))

  2%|▏         | 78/4800 [00:00<00:14, 325.71it/s]

{'eval_loss': 0.6968625783920288, 'eval_accuracy': 0.524, 'eval_f1': 0.0, 'eval_auc': 0.5, 'eval_precision': 0.0, 'eval_recall': 0.0, 'eval_runtime': 0.0155, 'eval_samples_per_second': 64467.254, 'eval_steps_per_second': 644.673, 'epoch': 1.0}


c:\Users\Ghiki\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))

  2%|▏         | 96/4800 [00:00<00:14, 325.71it/s]

{'eval_loss': 0.6920636892318726, 'eval_accuracy': 0.524, 'eval_f1': 0.0, 'eval_auc': 0.5, 'eval_precision': 0.0, 'eval_recall': 0.0, 'eval_runtime': 0.015, 'eval_samples_per_second': 66584.709, 'eval_steps_per_second': 665.847, 'epoch': 2.0}


  2%|▏         | 112/4800 [00:00<00:16, 284.16it/s]c:\Users\Ghiki\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))

  4%|▍         | 182/4800 [00:00<00:14, 311.03it/s]

...

100%|█████████▉| 4784/4800 [00:16<00:00, 297.68it/s]c:\Users\Ghiki\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))

100%|██████████| 4800/4800 [00:16<00:00, 297.68it/s]

{'eval_loss': 0.43678486347198486, 'eval_accuracy': 0.524, 'eval_f1': 0.0, 'eval_auc': 0.5, 'eval_precision': 0.0, 'eval_recall': 0.0, 'eval_runtime': 0.0176, 'eval_samples_per_second': 56868.834, 'eval_steps_per_second': 568.688, 'epoch': 100.0}


100%|██████████| 4800/4800 [00:16<00:00, 293.76it/s]

{'train_runtime': 16.3399, 'train_samples_per_second': 28984.246, 'train_steps_per_second': 293.759, 'train_loss': 0.4378258752822876, 'epoch': 100.0}


TrainOutput(global_step=4800, training_loss=0.4378258752822876, metrics={'train_runtime': 16.3399, 'train_samples_per_second': 28984.246, 'train_steps_per_second': 293.759, 'train_loss': 0.4378258752822876, 'epoch': 100.0})

In [30]:
trainer.evaluate()

  0%|          | 0/10 [00:00<?, ?it/s]c:\Users\Ghiki\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
100%|██████████| 10/10 [00:00<00:00, 454.03it/s]


{'eval_loss': 0.43568891286849976,
 'eval_accuracy': 0.524,
 'eval_f1': 0.0,
 'eval_auc': 0.5,
 'eval_precision': 0.0,
 'eval_recall': 0.0,
 'eval_runtime': 0.027,
 'eval_samples_per_second': 37009.0,
 'eval_steps_per_second': 370.09,
 'epoch': 100.0}

In [ ]:
trainer.evaluate(test_train_dataset)

In [ ]:
trainer.evaluate(test_dev_dataset)